In [3]:
# What is Arxiv
# Arxiv: arXiv is a free distribution service and an open-access archive for 
#nearly 2.4 million scholarly articles in the fields of physics, 
#mathematics, computer science, quantitative biology, quantitative finance, 
#statistics, electrical engineering and systems science, and economics.

In [4]:
# In this tutorial we will use the LangChain tools
# Link: https://python.langchain.com/v0.1/docs/integrations/tools/google_cloud_texttospeech/

Retrieve data from Wikipedia

In [7]:
# Wikipedia
from langchain_community.tools import WikipediaQueryRun
# Wrapper
from langchain_community.utilities import WikipediaAPIWrapper

In [10]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=400)

In [38]:
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)

In [39]:
wikipedia.name

'wikipedia'

Retrieve data from webpages

In [16]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [21]:
loader = WebBaseLoader(web_path="https://docs.smith.langchain.com/tutorials/optimize_classifier")
documents = loader.load()
documents_split = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64).split_documents(documents)

In [22]:
langchain_embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

c:\Users\Ronan\Documents\ML\LangChainProjects\LangChain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Ronan\Documents\ML\LangChainProjects\LangChain\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
vector_db = FAISS.from_documents(documents=documents_split, embedding=langchain_embed_model)

In [24]:
retriever = vector_db.as_retriever()

In [26]:
# We want to create a Retriever Tool
# Link: https://api.python.langchain.com/en/latest/tools/langchain.tools.retriever.create_retriever_tool.html
from langchain.tools.retriever import create_retriever_tool

In [29]:
retriever_tool = create_retriever_tool(retriever=retriever, 
                      name="LangChain_Classifier_Optimizer",
                      description="LangSmith retriever tool",
                      document_prompt="""Search for information about LangSmith to optimize classifiers.
                                         You will be able to search for information about LangSmith.
                                         This tool will be used for any information about LangSmith.
                                      """
                 )

In [30]:
retriever_tool.name

'LangChain_Classifier_Optimizer'

Retrieve data from Arxiv

In [31]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [36]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=400)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [37]:
arxiv.name

'arxiv'

In [40]:
tools = [
    wikipedia,
    arxiv,
    retriever_tool
]

In [41]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Ronan\\Documents\\ML\\LangChainProjects\\LangChain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=400)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=400, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='LangChain_Classifier_Optimizer', description='LangSmith retriever tool', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001DAEA3D0550>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_comm

In [42]:
# We want to query this multiple tools
# This is where we are going to use Agents
# Agents: 
# The core idea of agents is to use a language model to choose a sequence of 
#actions to take. 
# In chains, a sequence of actions is hardcoded (in code). 
# In agents, a language model is used as a reasoning engine to determine
#which actions to take and in which order.


In [73]:
from langchain_community.llms import Ollama
gemma = Ollama(model="gemma")

In [45]:
# Get the prompt to use from the hub
from langchain import hub

In [66]:
prompt = hub.pull("hwchase17/llama-rag")

In [71]:
from langchain.agents import create_tool_calling_agent
agent = initialize_agent(llama2, tools, prompt)

c:\Users\Ronan\Documents\ML\LangChainProjects\LangChain\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


TypeError: unhashable type: 'PromptTemplate'